# Gathering Data

Importing requried librarys

In [1]:
%%writefile ../src/datautil.py
#datautil.py
import os
import requests
import time
from bs4 import BeautifulSoup
import pandas as pd

Overwriting ../src/datautil.py


Functions for Downloading data

In [2]:
%%writefile -a ../src/datautil.py

def get_soup(url):
    
    
    response = requests.get(url)
    html = response.content
    soup = BeautifulSoup(html, 'html.parser')
    return soup

def download_data(url, n=50, sleep=1):
    cpus = [dict() for x in range(n)]
    soup = get_soup(url)
    table = soup.find('div', class_='chart')
    rows = table.find_all('li')
    attributes = ['Class:', 'Socket', 'Clocks', 'Turbo ', 'Cores:', 'Typica']

    i = 0
    for row in rows:
        link = row.find('a').get('href')
        link = 'https://www.cpubenchmark.net/' + link

        soup = get_soup(link)
        price = row.find('span', class_='price-neww').get_text()
        name = row.find('span', class_='prdname').get_text()
        

        cpus[i]["name"] = name
        cpus[i]["price"] = price

        table = soup.find('div', class_= 'left-desc-cpu')
        body = table.find_all('p')
        for col in body:
            text = col.get_text()
            title = col.find('strong').get_text()

            #Verifying attribute is one that we want
            if title[:6] in attributes:
                data = text.split(':')

                #Edge case for cores/threads being in the same exact <p> tag
                if title == "Cores:":

                    #Another edge case for amd cpu's
                    if (text.find("(") != -1):
                        data[1] = data[1][1]
                        data.append("Threads")
                        data.append(data[1])

                    else:
                        temp2 = data[2]
                        data[1] = data[1][1]
                        data[2] = "Threads"
                        data.append(temp2[1:])

                    cpus[i][data[2]] = data[3]

                ##Formatting strings
                if title != "Socket:":
                    data[1] = data[1].replace(" ", "")
                else:
                    data[1] = data[1][1:]

                if (title=='Clockspeed:') or (title=='Turbo Speed:'):
                    data[1] = data[1][:-3]

                #Assign to dict
                cpus[i][data[0]] = data[1]
                
        #Gathering scores from other part of dataset
        table = soup.find('div', class_= 'right-desc')
        body = table.find_all('p')
        multi_threaded_score = table.find_all('span')[1].get_text()
        
        data = table.get_text().split(":")
        data[1] = data[1][:5]
        data[1] = data[1].replace(" ", "")
        single_threaded_score = data[1]
        print(single_threaded_score)
        
        cpus[i]["mt_score"] = int(multi_threaded_score)
        cpus[i]["st_score"] = int(single_threaded_score)

        if i >= n-1:
            break
        i = i+1
        time.sleep(sleep)
    
    return pd.DataFrame.from_dict(cpus)

Appending to ../src/datautil.py


Adding local src folder to sys.path

In [3]:
import sys
project_dir = '/home/atoris/course-project-thomas-wright/src'
if project_dir not in sys.path:
    sys.path.insert(0, project_dir)


This next step actaully downloads all the data, this takes a while depnding on how fast you want to reload pages from the website, the rate limit is put into effect to not get blocked from the website.

In [6]:
import os
import pandas as pd
import datautil as du
import importlib

url = 'https://www.cpubenchmark.net/mid_range_cpus.html'

df = du.download_data(url, 100, 1)


1231
1981
2261
2072
1298
1965
2005
2051
1954
2101
2200
1078
1827
2247
2074
2039
2065
2010
1659
2239
1693
1091
1977
1565
1321
2258
2124
1127
2040
1856
1893
1777
1777
2005
1911
1932
1060
1189
1874
1875
1849
2123
569

1840
1157
1943
2012
1844
2200
2121
2062
2190
2186
1788
1165
1671
2052
971

2102
1954
2108
2141
1932
1766
1807
1776
1790
1164
2150
1956
1578
1734
1548
1698
1688
1997
1943
2064
1464
1694
2164
1753
1734
1896
1663
1916
1932
1737
1735
1725
1784
2081
849

2193
1726
1480
1782
1060
2308
2343


#### Verify the DataFrame

In [7]:
df.head(5)

,name,price,Class,Clockspeed,Turbo Speed,Threads,Cores,Typical TDP,mt_score,st_score,Socket
0,AMD Opteron 6344,$379.00*,Server,2.6,3.2,1,1,115W,6069,1231,NaN
1,Intel Core i5-8250U @ 1.60GHz,NA,Laptop,1.6,3.4,8,4,15W3,6067,1981,FC-BGA1356
2,Intel Core i5-6600 @ 3.30GHz,$262.13*,Desktop,3.3,3.9,4,4,65W,6059,2261,LGA 1151
3,Intel Xeon E3-1220 v6 @ 3.00GHz,$214.99,Server,3.0,3.5,4,4,72W,6048,2072,FCLGA1151
4,Intel Xeon E5-2440 @ 2.40GHz,$741.00*,Server,2.4,2.7,12,6,95W,6040,1298,LGA 1356


Functions for saving and reloading the dataframe

In [8]:
%%writefile -a ../src/datautil.py

def save_df(df, filename):
    basedir = os.path.dirname(os.getcwd())
    filepath = 'data/' + filename
    full_path = os.path.join(basedir, filepath)
    df.to_csv(full_path, index=False)

def load_data(url, file):
    file = os.path.dirname(os.getcwd()) + '/data/' + file
    print(file)
    if not os.path.exists(file):
        if not os.path.exists(os.path.dirname(file)):
            os.makedirs(os.path.dirname(file))
        df = download_data(url)
        save_df(df, file)
    return pd.read_csv(file)


Appending to ../src/datautil.py


Use these functions to save our data

In [9]:
importlib.reload(du)

du.save_df(df, 'cpu_data.csv')

Verify new project files have been saved

In [10]:
!tree ../

../
├── LICENSE
├── README.md
├── data
│   ├── cpu_data.csv
│   ├── cpu_data_cleaned.csv
│   └── cpu_data_encoded.csv
├── models
├── notebooks
│   ├── 0_setup_project_folders.ipynb
│   ├── 1_Retrieving_data.ipynb
│   ├── 2_Cleaning_data.ipynb
│   ├── 3_Simple_Graphs.ipynb
│   └── __pycache__
│       └── datautil.cpython-37.pyc
├── report
│   ├── interim.md
│   ├── proposal.md
│   └── report.md
└── src
    ├── __pycache__
    │   └── datautil.cpython-37.pyc
    └── datautil.py

7 directories, 15 files


Next step is to clean the rest of the data